# Import Lib

In [145]:
import pandas as pd 
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings


sys.path.append(r'C:\Users\antoi\Documents\Work_Learn\JEDHA\Function')
from function import describe_df



# EDA

In [146]:
df = pd.read_csv("Walmart_Store_sales.csv")
describe_df(df)

number of rows: 150
Display of dataset:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Basic statistics:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Pourcentage of missing values:


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [147]:
df['Store'].value_counts()
df['Store'].unique()

array([ 6., 13., 17., 11.,  4., 15., 20., 14.,  3.,  8., 18.,  7.,  1.,
        2.,  5., 19., 16., 12.,  9., 10.])

In [148]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day_of_week'] = df['Date'].dt.day_of_week
df = df.drop(['Date'],axis=1) 
df.dropna(subset=['Weekly_Sales'],inplace=True) #remove row when na is in weekly sales
len(df)

C:\Users\antoi\AppData\Local\Temp\ipykernel_23032\2170876047.py:1: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



136

In [149]:
df['day_of_week'].value_counts()/len(df)

day_of_week
4.0    0.867647
Name: count, dtype: float64

### Plot

In [150]:
import plotly.express as px
fig = px.histogram(df, x="year", nbins=3)
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="month", nbins=12)
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="day_of_week", nbins=7)
fig.update_layout(bargap=0.1)
fig.show()

fig = px.histogram(df, x="Holiday_Flag", nbins=2)
fig.update_layout(bargap=0.1)
fig.show()

fig = px.histogram(df, x="CPI",nbins=100)
fig.update_layout(bargap=0.1)
fig.show()

In [151]:
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1000, width = 1000)
fig.show()

In [152]:
corr_matrix = df.corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist()
                                  )

fig.update_layout(height=800, width = 800)

fig.show()

In [153]:
describe_df(df)

number of rows: 136
Display of dataset:


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day_of_week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,4.0


Basic statistics:


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day_of_week
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000,118.000000,118.000000,118.0
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582,2010.822034,6.338983,4.0
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428,0.812628,3.173664,0.0
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,2010.000000,1.000000,4.0
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000,2010.000000,4.000000,4.0
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000,2011.000000,6.000000,4.0
75%,15.250000,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000,2011.750000,9.000000,4.0
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000,2012.000000,12.000000,4.0


Pourcentage of missing values:


Store            0.000000
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
year            13.235294
month           13.235294
day_of_week     13.235294
dtype: float64

In [154]:
df.nunique().sort_values()


day_of_week       1
Holiday_Flag      2
year              3
month            12
Store            20
Unemployment    100
Fuel_Price      113
Temperature     119
CPI             123
Weekly_Sales    136
dtype: int64

# First Model

## Preprocessing

In [155]:
# numeric_features = []
# categorical_features = []
# for i,t in X.dtypes.items():
#     if ('float' in str(t)) or ('int' in str(t)) :
#         numeric_features.append(i)
#     else :
#         categorical_features.append(i)

# print('Found numeric features ', numeric_features)
# print('Found categorical features ', categorical_features)

# X.select_dtypes(include='number')# Automatically detect names of numeric/categorical columns
# numeric_features = X.select_dtypes(include='number').columns
# categorical_features = X.select_dtypes(exclude='number').columns

# print('Found numeric features ', numeric_features)
# print('Found categorical features ', categorical_features)

In [156]:
from sklearn.impute import KNNImputer

target = 'Weekly_Sales'
numeric_features = ['Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

categorical_features = ['Store']

df.dropna(subset=[target], inplace=True)
# features = df.columns.drop(target,axis=1)



Y = df[target]
X = df[numeric_features]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first'))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))

ValueError: A given column is not a column of the dataframe

In [120]:
coefs = pd.DataFrame(index = X_train.columns, data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

# Second model feature eng 

## V1

In [121]:
X.columns

Index(['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment'],
      dtype='object')

In [122]:

target = 'Weekly_Sales'
features = ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

X_eng = X.copy()

for col in X.columns:
    X_eng[col + '_2'] = X[col] ** 2
    X_eng[col + '_3'] = X[col] ** 3
    X_eng[col + '_4'] = X[col] ** 4
    X_eng[col + '_inverse'] = 1 / (X[col] + 1e-5)
    X_eng[col + '_inverse2'] = 1 / (X[col] ** 2 + 1e-5)

# X_eng = X_eng.drop(target, axis=1)
all_features = X_eng.columns



numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, all_features),
    ]
)

X_train, X_test, Y_train, Y_test = train_test_split(X_eng, Y, test_size=0.2, random_state=0)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))


coefs = pd.DataFrame(index = all_features, data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

Accuracy on training set :  0.7076577582038118
Accuracy on test set :  0.160118791157023


In [128]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Lasso,Ridge


regressor = Lasso()

params = {
    # 'alpha': np.arange(0, 100)
    'alpha': [1, 2, 3, 5, 10, 20, 30]
}

best_lasso = GridSearchCV(regressor, param_grid=params, cv=5)
best_lasso.fit(X_train, Y_train)

print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)



Best hyperparameters :  {'alpha': 1}
Best R2 score :  -0.10745170204347446


c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.013e+13, tolerance: 3.552e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.544e+12, tolerance: 3.342e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.747e+12, tolerance: 3.381e+09

In [124]:
scores = cross_val_score(best_lasso, X_train, Y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())


c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.075e+12, tolerance: 2.053e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.375e+12, tolerance: 2.093e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.204e+12, tolerance: 2.283e+09

The cross-validated R2-score is :  0.049943312617727366
The standard deviation is :  0.03084280960614103


c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.996e+12, tolerance: 2.638e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.257e+12, tolerance: 2.244e+09

c:\Users\antoi\Documents\Work_Learn\JEDHA\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.173e+12, tolerance: 2.694e+09

In [125]:
# new_df = pd.concat([X,Y],axis=1)
# corr_matrix = new_df.corr().round(3)

# corr_list = []
# corr_list_flat = []

# for c in corr_matrix.columns:
#     for r, corr_val in corr_matrix[c].items():
#         if ((c != r) and abs(corr_val) > 0.9) or (c==target):
#                 corr_list.append((c, r))
#                 corr_list_flat.append(c)
# # corr_list
# corr_list_flat

In [126]:
import matplotlib.pyplot as plt

# Calculate correlation matrix
# corr_matrix = X.corr()

# Plot
# plt.figure(figsize=(10, 8))
# plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
# plt.colorbar(label='Correlation')
# plt.title('Correlation Matrix of Features')
# plt.xticks(range(len(corr_matrix)), corr_matrix.columns, rotation=90)
# plt.yticks(range(len(corr_matrix)), corr_matrix.columns)
# plt.tight_layout()
# plt.show()

## V2

# Lasso to select col